Notebook for producing Figure 4.

In [ ]:
from bayesbeat.model import GenericAnalyticGaussianBeam
from bayesbeat.data import get_data
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pathlib

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"

Load one of the ringdowns so we can use the time samples for producing the plot.

In [ ]:
index = 0
x_data, y_data, frequency, _ = get_data(data_file, index=index)
x_data = x_data[:10000]

Define the analytic model ($M_3$) with $T={1,3,5,7}$

In [ ]:
models = []
n_terms = [1, 3, 5, 7]
t = np.linspace(0, 5e2, int(1e4))
for n in n_terms:
    model = GenericAnalyticGaussianBeam(
        t,
        None,
        photodiode_gap=0.5e-3,
        photodiode_size=10.2e-3,
        beam_radius=3.3e-3,
        include_gap=True,
        equation_name=f"General_Equation_{n}_Terms",
        n_terms=n,
        compile=False,    # Disable compilation since we only call the model once
    )
    models.append(model)

Define the parameters for the simulated signal and generate the fits for each model

In [ ]:
fits = {}
theta = dict(
    a_1=2e-3,
    a_2=1e-3,
    a_scale=1.0,
    tau_1=55.0,
    tau_2=50.0,
    domega=0.2,
    dphi=np.pi / 4,
    x_offset=0.0,
    sigma_amp_noise=0.0,
    sigma_constant_noise=0.0,
    mean_constant_noise=0.0,
)
for n, model in zip(n_terms, models):
    fits[n] = model.signal_model(theta)

Produce fits with smaller amplitudes.

In [ ]:
fits_1 = {}
theta_1 = theta.copy()
theta_1["a_1"] *= 0.1
theta_1["a_2"] *= 0.1
for n, model in zip(n_terms, models):
    fits_1[n] = model.signal_model(theta_1)

Produce the figure

In [ ]:
figsize = plt.rcParams["figure.figsize"].copy()
figsize[0] *= 1.8
figsize[1] *= 1.5

ls = ["-", "--", "-.", ":",]

fig, axs = plt.subplots(2, 2, figsize=figsize, sharex=True, sharey=False)

for i, (key, y_fit) in enumerate(fits.items()):
    axs[0, 0].plot(t, y_fit, label=key, ls=ls[i])
    # axs[1, 0].plot(t, y_fit / fits[1])
    if key == 1:
        continue
    axs[1, 0].plot(t, np.abs(y_fit / fits[1]), color=f"C{i}", ls=ls[i])

for i, (key, y_fit) in enumerate(fits_1.items()):
    axs[0, 1].plot(t, y_fit, label=key, color=f"C{i}", ls=ls[i])
    if key == 1:
        continue
    axs[1, 1].plot(t, np.abs( y_fit / fits_1[1]), color=f"C{i}", ls=ls[i])


for ax in axs.flatten():
    # ax.set_xscale("log")
    ax.set_xlim(0, 300)
    pass

# axs[0, 0].set_yscale("log")
# axs[0, 1].set_yscale("log")

# Adjust the y-limits
axs[0, 0].set_ylim(0, 1)  
axs[0, 1].set_ylim(0, 0.1)  
axs[1, 0].set_ylim(axs[1, 0].get_ylim()[0], 1)
axs[1, 1].set_ylim(axs[1, 1].get_ylim()[0], 0.99)

# Add the axis labels
axs[1, 0].set_xlabel("$t$ [s]")
axs[1, 1].set_xlabel("$t$ [s]")
axs[0, 0].set_ylabel("$A$")
axs[0, 1].set_ylabel("$A$")
axs[1, 0].set_ylabel("$A / A_{T=1}$")
axs[1, 1].set_ylabel("$A / A_{T=1}$")

axs[0, 0].set_title(r"$A_1 = A_2 = 10^{-4}$")
axs[0, 1].set_title(r"$A_1 = A_2 = 10^{-3}$")

handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, ncol=4, title=r"Number of terms ($T$)", loc="center", bbox_to_anchor=(0.5, 0.05))

plt.tight_layout()
fig.savefig(outdir / f"example_model_3.{file_format}")